# Pandas 时间序列分析在金融中的应用

## 金融数据分析综合指南

---

### 📊 学习目标：

完成本教程后，您将能够：

1. 使用 pandas **加载和操作金融时间序列数据**
2. 使用 **DateTimeIndex** 进行高效的时间基础操作
3. **重新采样金融数据**到不同频率（每日、每周、每月）
4. 使用滚动窗口和扩展操作**计算金融指标**
5. 执行**时间偏移**以创建滞后变量和收益
6. **分析投资组合表现**和风险指标
7. 处理金融数据集中的**缺失数据和数据质量问题**

### 📈 实际应用：

- 投资组合表现分析
- 风险管理计算
- 技术指标计算
- 市场相关性分析
- 金融报告和仪表板

### 📚 先决条件：

- 基础 Python 知识
- 理解金融概念（股票、收益、波动率）
- 基础 pandas DataFrame 操作

---

**注意**：本教程使用来自雅虎财经的真实金融市场数据，涵盖股票、ETF、债券和加密货币。

## 1. 设置和数据加载

### 🎯 **您将在本节中学到什么：**

在这个基础部分，您将掌握设置 pandas 环境和正确加载金融数据的基本技能。这些是每位金融分析师都需要知道的构建模块。

**关键知识点：**
- **库导入**：了解哪些库对金融时间序列分析至关重要
- **日期时间处理**：为什么在金融中正确管理日期/时间至关重要
- **数据加载最佳实践**：如何高效正确地加载金融数据
- **索引配置**：为最优时间序列操作设置数据结构

**为什么这很重要：**
金融数据本质上是基于时间的。股票价格、交易量和市场指标都依赖于精确的时间。从一开始正确设置数据结构将为您节省后期的调试时间，并启用 pandas 提供的强大时间基础操作。

### 金融分析必备库

In [46]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.width', None)
pd.set_option('display.max_columns', 50)

print("📊 Financial Data Analysis with Pandas - Setup Complete")
print(f"📅 Analysis Date: {pd.Timestamp.now().strftime('%Y-%m-%d')}")
print(f"🐼 Pandas Version: {pd.__version__}")
print(f"🔢 NumPy Version: {np.__version__}")

📊 Financial Data Analysis with Pandas - Setup Complete
📅 Analysis Date: 2025-10-11
🐼 Pandas Version: 2.3.1
🔢 NumPy Version: 2.3.2


### 1.1 使用正确的 DateTime 索引加载金融数据

**🧠 核心概念：DateTimeIndex**

**DateTimeIndex** 是 pandas 专门用于处理时间序列数据的索引类型。可以将其视为一个超级充电的行索引，能够理解日期、时间和金融日历概念。

**为什么 DateTimeIndex 对金融至关重要：**
- **自动对齐**：当您合并数据集时，pandas 会自动按日期对齐它们
- **基于时间的切片**：您可以轻松获取“2023 年所有数据”或“2022 年第四季度”
- **内置日历逻辑**：理解工作日、月末和节假日
- **重采样能力**：用一个命令将每日数据转换为每周/每月数据
- **性能**：在时间索引数据上的操作速度更快

**加载金融数据的两种方法：**

1. **方法 1（推荐）**：使用 `parse_dates=True` 直接加载 DateTimeIndex
2. **方法 2（替代）**：先加载，然后转换日期并设置索引

让我们看看两种方法的实际操作：

In [47]:
# Method 1: Best practice - Load with datetime index directly
# This is the recommended approach for financial data
apple_stock = pd.read_csv('./AAPL_stock_data.csv', index_col='Date', parse_dates=True)

print("📈 Apple 股票数据已加载:")
print(f"   日期范围: {apple_stock.index[0].date()} to {apple_stock.index[-1].date()}")
print(f"   总交易日: {len(apple_stock)}")
print(f"   列: {list(apple_stock.columns)}")
print(f"   索引类型: {type(apple_stock.index)}")

# Display first few rows
apple_stock.head()

📈 Apple 股票数据已加载:
   日期范围: 2020-09-21 to 2025-09-17
   总交易日: 1254
   列: ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
   索引类型: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-09-21,101.687917,107.183774,100.287201,107.076775,195713800,0.0,0.0
2020-09-22,109.605834,109.780924,106.181871,108.759567,183055400,0.0,0.0
2020-09-23,108.574771,109.051400,103.857083,104.197540,150718700,0.0,0.0
2020-09-24,102.300718,107.242125,102.135357,105.267509,167743300,0.0,0.0
2020-09-25,105.471789,109.372390,104.732522,109.216751,149981400,0.0,0.0


In [48]:
# Method 2: Alternative approach - Convert after loading
# Sometimes necessary when working with problematic date formats

# Load without datetime parsing first
raw_data = pd.read_csv('./MSFT_stock_data.csv')
print("转换前:")
print(f"{raw_data.head(3).to_string()}")
print(f"前几个日期: {raw_data['Date'].head(3).tolist()}")

# Convert Date column to datetime and set as index
print("转换后:")
raw_data['Date'] = pd.to_datetime(raw_data['Date'])
raw_data = raw_data.set_index('Date')
print(f"{raw_data.head(3).to_string()}")

转换前:
         Date        Open        High         Low       Close    Volume  Dividends  Stock Splits
0  2020-09-21  189.070793  194.363514  188.294147  194.200500  39839700        0.0           0.0
1  2020-09-22  196.616734  199.531571  193.759438  198.879562  33517100        0.0           0.0
2  2020-09-23  199.339791  199.531568  191.793840  192.330780  30803800        0.0           0.0
前几个日期: ['2020-09-21', '2020-09-22', '2020-09-23']
转换后:
                  Open        High         Low       Close    Volume  Dividends  Stock Splits
Date                                                                                         
2020-09-21  189.070793  194.363514  188.294147  194.200500  39839700        0.0           0.0
2020-09-22  196.616734  199.531571  193.759438  198.879562  33517100        0.0           0.0
2020-09-23  199.339791  199.531568  191.793840  192.330780  30803800        0.0           0.0


### 1.2 理解金融数据结构

**📊 金融 OHLCV 数据剖析**

大多数金融数据集遵循 **OHLCV** 格式，它捕获每个交易周期的完整价格和交易量信息：

| 列 | 全称 | 代表含义 | 使用时机 |
|--------|-----------|-------------------|-------------|
| **Open** | 开盘价 | 周期的第一个交易价格 | 缺口分析，隔夜变动 |
| **High** | 最高价 | 周期内的峰值价格 | 阻力位，波动率 |
| **Low** | 最低价 | 周期内的最低价格 | 支撑位，风险评估 |
| **Close** | 收盘价 | 周期的最后一个交易价格 | **分析中最常用** |
| **Volume** | 交易量 | 交易的股票/合约数量 | 流动性，验证变动 |
| **Adj Close** | 调整后收盘价 | 调整过分红/拆股的收盘价 | **收益计算最佳** |

**💡 专业提示：**
- **大多数分析使用 'Close'** - 它是标准参考价格
- **收益计算使用 'Adj Close'** - 考虑公司行动（股票拆分、分红）
- **交易量确认价格变动** - 高交易量 = 更可靠的价格变化
- **高低范围衡量波动率** - 更宽范围表示更多不确定性

让我们探索数据集结构：

In [49]:
# Explore the financial data structure
print("📊 APPLE 股票数据概览")
print("=" * 40)

# Basic information about the dataset
print("\n📋 数据集信息:")
apple_stock.info()

print("\n📊 统计摘要:")
print(apple_stock.describe().round(2))

print("\n🗓️ 日期范围分析：")
print(f"开始日期：{apple_stock.index.min()}")
print(f"结束日期：{apple_stock.index.max()}")
print(f"交易日：{len(apple_stock)}")
print(f"日期范围：{(apple_stock.index.max() - apple_stock.index.min()).days} 个日历日")

# Check for missing values
print("\n❓ 缺失值：")
missing_values = apple_stock.isnull().sum()
if missing_values.sum() == 0:
    print("✅ 未发现缺失值")
else:
    print(missing_values[missing_values > 0])

📊 APPLE 股票数据概览

📋 数据集信息:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1254 entries, 2020-09-21 to 2025-09-17
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          1254 non-null   float64
 1   High          1254 non-null   float64
 2   Low           1254 non-null   float64
 3   Close         1254 non-null   float64
 4   Volume        1254 non-null   int64  
 5   Dividends     1254 non-null   float64
 6   Stock Splits  1254 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 78.4 KB

📊 统计摘要:
          Open     High      Low    Close        Volume  Dividends  \
count  1254.00  1254.00  1254.00  1254.00  1.254000e+03    1254.00   
mean    170.84   172.73   169.11   171.01  7.391611e+07       0.00   
std      36.33    36.58    36.12    36.39  3.247052e+07       0.03   
min     101.69   107.18   100.29   104.20  2.323470e+07       0.00   
25%     142.44   144.38   140.84   142.79  5.052690e+0

### 1.3 使用 Pandas 进行基于时间的数据切片

**🎯 核心概念**

在使用 pandas 处理时间序列数据时，您经常需要提取特定日期范围或按时间段过滤数据。现代 pandas 需要**完整的日期规范**或**布尔索引**以实现可靠的时间基础切片。

**🔄 切片方法总结**

| 方法 | 语法 | 使用案例 | 示例 |
|--------|--------|----------|---------|
| **完整日期范围** | `df.loc['start':'end']` | 精确日期范围 | `apple_stock.loc['2024-01-01':'2024-01-31']` |
| **布尔索引** | `df[df.index.condition]` | 灵活过滤 | `apple_stock[apple_stock.index.year == 2024]` |
| **日期偏移** | `df.loc[date_offset:]` | 相对期间 | `apple_stock.loc[six_months_ago:]` |
| **查询方法** | `df.query('condition')` | 可读语法 | `apple_stock.query('index.year == 2024')` |
| **交集** | `df.loc[dates.intersection()]` | 处理缺失日期 | `apple_stock.loc[business_days.intersection(apple_stock.index)]` |

**基本切片技术：**

1. **完整日期范围切片** - 使用完整日期字符串与 `.loc[]`
2. **布尔索引** - 使用日期时间组件进行灵活过滤
3. **日期偏移操作** - 使用 pandas 日期偏移进行相对期间
4. **查询方法** - 日期过滤的替代可读方法

**⚠️ 重要说明：**
- 现代 pandas **不支持**部分字符串索引，如 `df['2024']` 或 `df['2024-01']`
- 始终使用**完整日期**或**布尔索引**以获得可靠结果
- 使用 `&`（与）和 `|`（或）运算符组合条件
- 组合多个布尔条件时使用括号

In [50]:
# Demonstrate various slicing techniques
print("📊 PANDAS 基于时间的切片示例")
print("=" * 40)

# 1. Complete Date Range Slicing
print("\n1️⃣ 完整日期范围切片：")

# Single specific date (if exists)
single_date = apple_stock.loc['2024-01-03']
print(f"✅ 单日访问：2024-01-03")
print(f"收盘价：${single_date['Close']:.2f}")

multi_dates = apple_stock.loc['2024-01-03':'2025-01-10']
print("从 2024-01-03 到 2025-01-10 的总天数：", len(multi_dates))


# Date range (first 10 days of available data)
print(f"\n✅ 前 10 个交易日：")
first_10_days = apple_stock.head(10)
print(f"日期范围：{first_10_days.index[0].strftime('%Y-%m-%d')} 至 {first_10_days.index[-1].strftime('%Y-%m-%d')}")
print(f"交易日数量：{len(first_10_days)}")

# 2. Boolean Indexing (Recommended Approach)
print("\n2️⃣ 布尔索引：")

# Filter by year
available_years = apple_stock.index.year.unique()
target_year = available_years[-1]  # Use the most recent year
year_data = apple_stock[apple_stock.index.year == target_year]
print(f"✅ {target_year} 年：{len(year_data)} 个交易日")

# Filter by month and year
if len(year_data) > 0:
    target_month = year_data.index[0].month
    month_data = apple_stock[(apple_stock.index.year == target_year) & 
                           (apple_stock.index.month == target_month)]
    print(f"✅ {target_year}-{target_month:02d}：{len(month_data)} 个交易日")

# Multiple years
if len(available_years) >= 2:
    recent_years = available_years[-2:]
    multi_year = apple_stock[apple_stock.index.year.isin(recent_years)]
    print(f"✅ {recent_years[0]}-{recent_years[1]} 年：{len(multi_year)} 个交易日")

# 3. Date Offset Operations
print("\n3️⃣ 日期偏移操作：")

# Last 30 calendar days
thirty_days_ago = apple_stock.index[-1] - pd.Timedelta(days=30)
last_30_days = apple_stock.loc[thirty_days_ago:]
print(f"✅ 过去 30 天：{len(last_30_days)} 个交易日")
print(f"日期范围：{last_30_days.index[0].strftime('%Y-%m-%d')} 至 {last_30_days.index[-1].strftime('%Y-%m-%d')}")

# Last 6 months using DateOffset
six_months_ago = apple_stock.index[-1] - pd.DateOffset(months=6)
last_6_months = apple_stock.loc[six_months_ago:]
print(f"✅ 过去 6 个月：{len(last_6_months)} 个交易日")

# 4. Query Method
print("\n4️⃣ 查询方法：")
query_result = apple_stock.query('index > "2025-01-01"')
print(f"✅ 查询方法获取 {target_year} 年：{len(query_result)} 个交易日")
query_result = apple_stock.query('index > "2025-02"')
print(f"✅ 查询方法获取 {target_year} 年 2 月起：{len(query_result)} 个交易日")

# Practical Example: Quarterly Analysis
print("\n📈 实际示例：季度分析")
print("=" * 35)

# Find a year with sufficient data
for year in reversed(available_years):
    year_subset = apple_stock[apple_stock.index.year == year]
    if len(year_subset) > 100:  # Ensure we have substantial data
        q1_data = year_subset[year_subset.index.month.isin([1, 2, 3])]
        q4_data = year_subset[year_subset.index.month.isin([10, 11, 12])]

        if len(q1_data) > 0 and len(q4_data) > 0:
            print(f"\n📊 {year} 年季度分析：")
            print(f"Q1 交易日：{len(q1_data)}")
            print(f"Q4 交易日：{len(q4_data)}")

            # Calculate quarterly returns
            q1_return = (q1_data['Close'].iloc[-1] / q1_data['Close'].iloc[0] - 1) * 100
            q4_return = (q4_data['Close'].iloc[-1] / q4_data['Close'].iloc[0] - 1) * 100

            print(f"Q1 收益：{q1_return:.2f}%")
            print(f"Q4 收益：{q4_return:.2f}%")


print("\n✅ 所有切片示例成功完成！")

📊 PANDAS 基于时间的切片示例

1️⃣ 完整日期范围切片：
✅ 单日访问：2024-01-03
收盘价：$182.70
从 2024-01-03 到 2025-01-10 的总天数： 257

✅ 前 10 个交易日：
日期范围：2020-09-21 至 2020-10-02
交易日数量：10

2️⃣ 布尔索引：
✅ 2025 年：177 个交易日
✅ 2025-01：20 个交易日
✅ 2024-2025 年：429 个交易日

3️⃣ 日期偏移操作：
✅ 过去 30 天：22 个交易日
日期范围：2025-08-18 至 2025-09-17
✅ 过去 6 个月：128 个交易日

4️⃣ 查询方法：
✅ 查询方法获取 2025 年：177 个交易日
✅ 查询方法获取 2025 年 2 月起：157 个交易日

📈 实际示例：季度分析

📊 2024 年季度分析：
Q1 交易日：61
Q4 交易日：64
Q1 收益：-7.51%
Q4 收益：10.82%

📊 2023 年季度分析：
Q1 交易日：62
Q4 交易日：63
Q1 收益：32.05%
Q4 收益：10.95%

📊 2022 年季度分析：
Q1 交易日：62
Q4 交易日：63
Q1 收益：-3.94%
Q4 收益：-8.64%

📊 2021 年季度分析：
Q1 交易日：61
Q4 交易日：64
Q1 收益：-5.47%
Q4 收益：24.66%

✅ 所有切片示例成功完成！


## 2 列选择和多列操作

**🎯 处理金融数据列**

在金融分析中，您经常需要处理特定列或列的组合。以下是最常见的模式：

**常见选择模式：**
- **单列**：`data['Close']` → 返回 pandas Series
- **多列**：`data[['Open', 'Close']]` → 返回 DataFrame
- **OHLC 子集**：`data[['Open', 'High', 'Low', 'Close']]` → 传统价格数据
- **价格 + 交易量**：`data[['Close', 'Volume']]` → 价格和流动性分析

**💡 Series 与 DataFrame：**
- **Series**（单列）：非常适合计算、绘制单个指标
- **DataFrame**（多列）：需要进行比较、相关性、多资产分析

**实际应用：**
- **OHLC 分析**：计算每日范围、缺口、价格模式
- **交易量分析**：确认价格变动，识别积累/分配
- **价格转换**：从现有列创建新指标

让我们看看这些概念的实际操作：

In [51]:
# Working with specific columns
print("📊 列选择和操作")
print("=" * 35)

# Close prices time series
close_prices = apple_stock['Close']
print(f"\n📈 收盘价序列：")
print(f"类型：{type(close_prices)}")
print(f"长度：{len(close_prices)}")
print(f"最新价格：${close_prices.iloc[-1]:.2f}")

# OHLC data (Open, High, Low, Close)
ohlc_data = apple_stock[['Open', 'High', 'Low', 'Close']]
print(f"\n📊 OHLC 数据形状：{ohlc_data.shape}")
print("\n最近 3 个交易日 OHLC：")
print(ohlc_data.tail(3).round(2))

# Price and volume analysis
price_volume = apple_stock[['Close', 'Volume']]
print(f"\n💰 近期价格 vs 交易量：")
recent_pv = price_volume.tail(5)
recent_pv['Volume_Million'] = recent_pv['Volume'] / 1_000_000
print(recent_pv[['Close', 'Volume_Million']].round(2))

📊 列选择和操作

📈 收盘价序列：
类型：<class 'pandas.core.series.Series'>
长度：1254
最新价格：$238.99

📊 OHLC 数据形状：(1254, 4)

最近 3 个交易日 OHLC：
              Open    High     Low   Close
Date                                      
2025-09-15  237.00  238.19  235.03  236.70
2025-09-16  237.18  241.22  236.32  238.15
2025-09-17  238.97  240.10  237.73  238.99

💰 近期价格 vs 交易量：
             Close  Volume_Million
Date                              
2025-09-11  230.03           50.21
2025-09-12  234.07           55.82
2025-09-15  236.70           42.70
2025-09-16  238.15           63.42
2025-09-17  238.99           46.44


## 3. 金融计算和衍生指标

这是 pandas 成为强大金融分析工具的地方。您将学习将原始价格数据转换为有意义的金融指标，从而驱动投资决策。

**关键知识点：**
- **收益计算**：所有金融分析的基础
- **年化**：将每日指标转换为年度等效值
- **风险指标**：波动率、夏普比率和其他风险度量
- **技术指标**：移动平均线、动量指标和趋势分析

**为什么这很重要：**
原始价格告诉我们很少关于投资表现的信息。重要的是**收益**（您赚多少钱）、**波动率**（风险有多大）和**风险调整后收益**（每单位风险获得多少收益）。这些计算构成了投资组合管理、风险评估和交易策略的基础。

### 3.1 收益计算

**📈 理解金融收益**

收益是价格随时间变化的百分比 - 这是金融的基本构建块。不同类型的收益服务于不同的目的：

| 收益类型 | 公式 | 使用时机 | 关键优势 |
|-------------|---------|-------------|--------------|
| **简单收益** | `(P₁ - P₀) / P₀` | 表现测量 | 易于解释 |
| **对数收益** | `ln(P₁ / P₀)` | 风险建模，时间序列 | 随时间可加 |
| **周期收益** | 相同公式，不同周期 | 比较不同时间框架 | 标准化比较 |

**🕐 金融中的时间范围：**
- **每日**：日间价格变化，短期交易
- **每周**：减少噪音，中期模式
- **每月**：战略分析，长期趋势
- **年度**：表现评估，基准测试

**💡 专业提示：**
- 使用 `pct_change()` 计算简单收益
- 使用 `pct_change(periods=5)` 计算周收益（5 个交易日）
- 始终乘以 100 以百分比显示
- 对数收益更适合风险计算

收益是衡量表现和风险的金融基础。

In [52]:
# Calculate various types of returns
print("📊 金融收益计算")
print("=" * 35)

# Daily returns (percentage change)
apple_stock['Daily_Return'] = apple_stock['Close'].pct_change()
apple_stock['Daily_Return_Pct'] = apple_stock['Daily_Return'] * 100

# Weekly returns (comparing to 5 trading days ago)
apple_stock['Weekly_Return'] = apple_stock['Close'].pct_change(periods=5)
apple_stock['Weekly_Return_Pct'] = apple_stock['Weekly_Return'] * 100

# Monthly returns (comparing to ~21 trading days ago)
apple_stock['Monthly_Return'] = apple_stock['Close'].pct_change(periods=21)
apple_stock['Monthly_Return_Pct'] = apple_stock['Monthly_Return'] * 100

# Log returns (for risk calculations)
apple_stock['Log_Return'] = np.log(apple_stock['Close'] / apple_stock['Close'].shift(1))

print("✅ 收益计算成功！")
print("\n📊 收益摘要（最近 10 天）：")
returns_summary = apple_stock[['Close', 'Daily_Return_Pct', 'Weekly_Return_Pct']].tail(10)
print(returns_summary.round(3))

# Returns statistics
print("\n📈 日收益统计：")
daily_stats = apple_stock['Daily_Return_Pct'].describe()
print(f"均值：{daily_stats['mean']:.3f}%")
print(f"标准差：{daily_stats['std']:.3f}%")
print(f"最小值：{daily_stats['min']:.3f}%")
print(f"最大值：{daily_stats['max']:.3f}%")

# Annualized metrics
trading_days = 252
annual_return = apple_stock['Daily_Return'].mean() * trading_days * 100
annual_volatility = apple_stock['Daily_Return'].std() * np.sqrt(trading_days) * 100
sharpe_ratio = annual_return / annual_volatility

print(f"\n📊 年化指标：")
print(f"年化收益：{annual_return:.2f}%")
print(f"年化波动率：{annual_volatility:.2f}%")
print(f"夏普比率：{sharpe_ratio:.3f}")

📊 金融收益计算
✅ 收益计算成功！

📊 收益摘要（最近 10 天）：
             Close  Daily_Return_Pct  Weekly_Return_Pct
Date                                                   
2025-09-04  239.78             0.549              4.031
2025-09-05  239.69            -0.038              3.066
2025-09-08  237.88            -0.755              2.473
2025-09-09  234.35            -1.484              2.015
2025-09-10  226.79            -3.226             -4.898
2025-09-11  230.03             1.429             -4.066
2025-09-12  234.07             1.756             -2.345
2025-09-15  236.70             1.124             -0.496
2025-09-16  238.15             0.613              1.622
2025-09-17  238.99             0.353              5.379

📈 日收益统计：
均值：0.080%
标准差：1.809%
最小值：-9.246%
最大值：15.329%

📊 年化指标：
年化收益：20.26%
年化波动率：28.72%
夏普比率：0.705


### 3.2 使用滚动窗口的技术指标

**📊 使用 Pandas 进行技术分析**

技术指标是基于历史价格和交易量数据的数学计算。它们帮助识别趋势、动量和潜在反转点。Pandas 的 `rolling()` 函数使计算这些指标变得简单。

**🔧 核心概念：**

**滚动窗口**：一个“移动”计算，在数据中滑动
- `rolling(window=20)` = 最近 20 个观测值
- 每个点使用最近 N 个观测值
- 创建平滑的、跟随趋势的指标

**技术指标类型：**

| 类别 | 示例 | 目的 | Pandas 实现 |
|----------|----------|---------|----------------------|
| **趋势跟踪** | SMA, EMA | 识别方向 | `rolling().mean()`, `ewm()` |
| **动量** | RSI, MACD | 衡量变化速度 | 自定义计算 |
| **波动率** | 布林带 | 衡量价格变化 | `rolling().std()` |
| **交易量** | 交易量移动平均 | 确认价格变动 | `rolling().mean()` 应用于交易量 |

**📈 初学者最重要的指标：**

1. **简单移动平均线 (SMA)**：N 个周期的平均价格
2. **指数移动平均线 (EMA)**：加权平均，偏好近期价格
3. **布林带**：基于标准差的价格通道
4. **RSI**：动量振荡器（0-100 尺度）
5. **MACD**：趋势和动量组合

让我们逐步实现这些：

In [53]:
# Calculate technical indicators
print("📊 技术指标计算")
print("=" * 38)

# Simple Moving Averages
apple_stock['SMA_10'] = apple_stock['Close'].rolling(window=10).mean()
apple_stock['SMA_20'] = apple_stock['Close'].rolling(window=20).mean()
apple_stock['SMA_50'] = apple_stock['Close'].rolling(window=50).mean()
apple_stock['SMA_200'] = apple_stock['Close'].rolling(window=200).mean()

# Exponential Moving Average
apple_stock['EMA_12'] = apple_stock['Close'].ewm(span=12).mean()
apple_stock['EMA_26'] = apple_stock['Close'].ewm(span=26).mean()

# MACD (Moving Average Convergence Divergence)
apple_stock['MACD'] = apple_stock['EMA_12'] - apple_stock['EMA_26']
apple_stock['MACD_Signal'] = apple_stock['MACD'].ewm(span=9).mean()
apple_stock['MACD_Histogram'] = apple_stock['MACD'] - apple_stock['MACD_Signal']

# Bollinger Bands
apple_stock['BB_Middle'] = apple_stock['Close'].rolling(window=20).mean()
bb_std = apple_stock['Close'].rolling(window=20).std()
apple_stock['BB_Upper'] = apple_stock['BB_Middle'] + (bb_std * 2)
apple_stock['BB_Lower'] = apple_stock['BB_Middle'] - (bb_std * 2)
apple_stock['BB_Width'] = apple_stock['BB_Upper'] - apple_stock['BB_Lower']
apple_stock['BB_Position'] = (apple_stock['Close'] - apple_stock['BB_Lower']) / apple_stock['BB_Width']

# Relative Strength Index (RSI) - simplified version
delta = apple_stock['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
apple_stock['RSI'] = 100 - (100 / (1 + rs))

# Volatility (rolling standard deviation)
apple_stock['Volatility_10d'] = apple_stock['Daily_Return'].rolling(window=10).std() * np.sqrt(252) * 100
apple_stock['Volatility_30d'] = apple_stock['Daily_Return'].rolling(window=30).std() * np.sqrt(252) * 100

print("✅ 技术指标计算成功！")
print("\n📊 最新技术指标：")
latest_indicators = apple_stock[[
    'Close', 'SMA_20', 'SMA_50', 'RSI', 'MACD', 'BB_Position', 'Volatility_30d'
]].tail(5)
print(latest_indicators.round(3).to_string())

📊 技术指标计算
✅ 技术指标计算成功！

📊 最新技术指标：
             Close   SMA_20   SMA_50     RSI   MACD  BB_Position  Volatility_30d
Date                                                                            
2025-09-11  230.03  231.643  220.564  56.752  3.763        0.409          29.676
2025-09-12  234.07  231.707  221.001  58.055  3.624        0.632          29.779
2025-09-15  236.70  231.963  221.469  61.578  3.684        0.756          28.514
2025-09-16  238.15  232.326  222.038  60.914  3.804        0.803          28.514
2025-09-17  238.99  232.747  222.622  60.583  3.922        0.811          28.430


### 3.3 技术指标摘要

In [54]:
# Display recent price data with moving averages for analysis
recent_data = apple_stock.tail(10)  # Last 10 days of data

print("📊 近期价格数据与移动平均线：")
price_ma_summary = recent_data[['Close', 'SMA_10', 'SMA_20', 'SMA_50']].round(2)
print(price_ma_summary)

# Display Bollinger Bands analysis
print("\n📊 布林带（Bollinger Bands）分析（最近 5 天）：")
bb_analysis = recent_data[['Close', 'BB_Lower', 'BB_Middle', 'BB_Upper', 'BB_Position']].round(3)
print(bb_analysis.tail())

# Display RSI and MACD indicators
print("\n📊 技术指标（Technical Indicators）（最近 5 天）：")
tech_indicators = recent_data[['Close', 'RSI', 'MACD', 'MACD_Signal', 'MACD_Histogram']].round(3)
print(tech_indicators.tail())

# Returns distribution summary
returns_clean = apple_stock['Daily_Return_Pct'].dropna()
print("\n📊 日收益分布摘要：")
print(f"均值：{returns_clean.mean():.3f}%")
print(f"标准差：{returns_clean.std():.3f}%")
print(f"最小值：{returns_clean.min():.3f}%")
print(f"最大值：{returns_clean.max():.3f}%")
print(f"25th 百分位数：{returns_clean.quantile(0.25):.3f}%")
print(f"75th 百分位数：{returns_clean.quantile(0.75):.3f}%")

print("\n✅ 技术指标分析完成！")


📊 近期价格数据与移动平均线：
             Close  SMA_10  SMA_20  SMA_50
Date                                      
2025-09-04  239.78  231.23  230.16  217.50
2025-09-05  239.69  232.71  231.15  218.27
2025-09-08  237.88  233.72  231.59  219.01
2025-09-09  234.35  234.44  231.95  219.68
2025-09-10  226.79  234.19  231.81  220.11
2025-09-11  230.03  234.14  231.64  220.56
2025-09-12  234.07  234.29  231.71  221.00
2025-09-15  236.70  234.75  231.96  221.47
2025-09-16  238.15  235.59  232.33  222.04
2025-09-17  238.99  235.64  232.75  222.62

📊 布林带（Bollinger Bands）分析（最近 5 天）：
             Close  BB_Lower  BB_Middle  BB_Upper  BB_Position
Date                                                          
2025-09-11  230.03   222.735    231.643   240.551        0.409
2025-09-12  234.07   222.746    231.707   240.669        0.632
2025-09-15  236.70   222.729    231.963   241.197        0.756
2025-09-16  238.15   222.707    232.326   241.945        0.803
2025-09-17  238.99   222.724    232.747   242.771      

## 4. 时间序列重采样

### 🎯 **您将在本节中学到什么：**

重采样是 pandas 在金融分析中最强大的功能之一。您将学习在不同时间频率之间转换数据 - 这是多时间框架分析和报告的关键技能。

**关键知识点：**
- **频率转换**：将每日数据转换为每周、每月或每年
- **聚合方法**：为不同类型的数据选择正确的方法
- **工作日历感知**：处理交易日与日历日
- **性能影响**：重采样如何影响您的分析

**实际应用：**
- **季度报告**：将日收益转换为季度表现
- **风险管理**：从日数据计算月波动率
- **策略测试**：在不同时间框架测试交易策略
- **基准测试**：使用标准化期间比较表现

**🕰️ 金融时间频率：**

| 频率 | 代码 | 使用案例 | 典型分析 |
|-----------|------|----------|------------------|
| **每日** | 'D' | 日内交易，短期 | 日内模式，日收益 |
| **每周** | 'W' | 波段交易 | 周趋势，减少噪音 |
| **每月** | 'M' | 投资组合管理 | 月度报告，长期趋势 |
| **季度** | 'Q' | 基本面分析 | 盈利期，战略规划 |
| **年度** | 'A' | 投资规划 | 年度表现，基准测试 |

重采样允许您更改时间序列数据的频率 - 对于不同时间范围的金融分析至关重要。

### 4.1 频率转换

**🔄 理解重采样逻辑**

当您重采样金融数据时，您本质上是在问：*“不要每日观测，而是显示每周/每月/年度摘要。”* 但什么值代表每个新周期？

**聚合决策：**

对于**价格数据**，您通常需要**最后**值（周期的收盘价）：
- `resample('W').last()` → 周收盘价
- `resample('M').last()` → 月末价格

对于**交易量数据**，您通常需要**总和**（总交易活动）：
- `resample('W').sum()` → 周总交易量
- `resample('M').sum()` → 月总交易量

**🎯 常见重采样模式：**

| 数据类型 | 最佳聚合 | 原因 |
|-----------|------------------|--------|
| **价格** | `.last()` | 收盘价代表该周期 |
| **交易量** | `.sum()` | 总交易活动 |
| **收益** | `.sum()` | 周期内的累计收益 |
| **波动率** | `.mean()` | 平均波动率水平 |

**⚠️ 重要说明：**
- 更高频率会丢失信息（每日 → 每月丢失每日变化）
- 更低频率平滑噪音并揭示长期趋势
- 根据分析目标选择频率

让我们看看重采样的实际操作：

In [55]:
# Load multi-asset data for resampling examples
portfolio_data = pd.read_csv('sample_portfolio.csv', index_col='Date', parse_dates=True)

print("📊 时间序列重采样")
print("=" * 28)
print(f"\n📈 原始投资组合数据：{portfolio_data.shape}")
print(f"频率：每日数据")
print(f"日期范围：{portfolio_data.index[0].date()} 至 {portfolio_data.index[-1].date()}")

# Common resampling frequencies in finance
frequency_map = {
      'Weekly': 'W',      # Week ending
      'Monthly': 'M',     # Month ending (older pandas)
      'Quarterly': 'Q',   # Quarter ending (older pandas)
      'Yearly': 'A'       # Year ending (older pandas)
  }

print("\n📊 将投资组合价值重采样到不同频率：")
resampled_data = {}

for freq_name, freq_code in frequency_map.items():
    # For price data, we typically want the last value (last trading price)
    resampled = portfolio_data['Portfolio_Value'].resample(freq_code).last()
    resampled_data[freq_name] = resampled
    
    print(f"\n{freq_name} ({freq_code})：{len(resampled)} 个周期")
    print(f"最新值：")
    print(resampled.tail(3).round(2))

# Calculate returns at different frequencies
print("\n📈 不同频率的收益：")
for freq_name, data in resampled_data.items():
    returns = data.pct_change().dropna() * 100
    print(f"\n{freq_name} 收益（最近 5 个）：")
    print(f"均值：{returns.mean():.2f}%，标准差：{returns.std():.2f}%")
    print(returns.tail(5).round(2))

📊 时间序列重采样

📈 原始投资组合数据：(1254, 12)
频率：每日数据
日期范围：2020-09-21 至 2025-09-17

📊 将投资组合价值重采样到不同频率：

Weekly (W)：261 个周期
最新值：
Date
2025-09-07    273.21
2025-09-14    281.74
2025-09-21    288.93
Freq: W-SUN, Name: Portfolio_Value, dtype: float64

Monthly (M)：61 个周期
最新值：
Date
2025-07-31    247.12
2025-08-31    264.18
2025-09-30    288.93
Freq: ME, Name: Portfolio_Value, dtype: float64

Quarterly (Q)：21 个周期
最新值：
Date
2025-03-31    212.06
2025-06-30    236.14
2025-09-30    288.93
Freq: QE-DEC, Name: Portfolio_Value, dtype: float64

Yearly (A)：6 个周期
最新值：
Date
2023-12-31    185.92
2024-12-31    242.55
2025-12-31    288.93
Freq: YE-DEC, Name: Portfolio_Value, dtype: float64

📈 不同频率的收益：

Weekly 收益（最近 5 个）：
均值：0.46%，标准差：3.22%
Date
2025-08-24    0.31
2025-08-31    0.59
2025-09-07    3.41
2025-09-14    3.12
2025-09-21    2.55
Freq: W-SUN, Name: Portfolio_Value, dtype: float64

Monthly 收益（最近 5 个）：
均值：1.95%，标准差：6.42%
Date
2025-05-31    8.36
2025-06-30    0.87
2025-07-31    4.65
2025-08-31    6.91
2025-09-30  

### 4.2 金融数据的聚合方法

**🎯 高级聚合策略**

金融数据中的不同列需要不同的聚合方法。`agg()` 方法允许您指定每列应如何汇总：

**OHLCV 聚合规则：**

| 列 | 聚合 | 逻辑 |
|--------|-------------|-------|
| **Open** | `first` | 周期的第一个价格 |
| **High** | `max` | 周期内的最高价格 |
| **Low** | `min` | 周期内的最低价格 |
| **Close** | `last` | 周期的最终价格 |
| **Volume** | `sum` | 总交易股票数 |

**📊 多指标聚合：**

您可以为同一列计算多个统计量：
```python
.agg({
    'Close': ['first', 'last', 'min', 'max', 'mean'],
    'Volume': ['sum', 'mean', 'std']
})
```

**💡 为什么这很重要：**
- **OHLC 保留**：在任何频率下保持基本价格结构
- **统计丰富性**：在一个操作中获得全面摘要
- **比较分析**：比较日内与周期结束值

让我们实现专业级聚合：

In [56]:
print("📊 金融数据聚合方法")
print("=" * 42)

# Select OHLCV data for aggregation
ohlcv_data = apple_stock[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()

# Weekly aggregation with appropriate methods for each column
weekly_ohlc = ohlcv_data.resample('W').agg({
    'Open': 'first',    # First trading day's opening price
    'High': 'max',      # Highest price during the week
    'Low': 'min',       # Lowest price during the week
    'Close': 'last',    # Last trading day's closing price
    'Volume': 'sum'     # Total volume traded during the week
})

print("\n📅 每周 OHLCV 数据（最近 8 周）：")
print(weekly_ohlc.tail(8).round(2))

# Monthly aggregation
monthly_stats = ohlcv_data.resample('M').agg({
    'Close': ['first', 'last', 'min', 'max', 'mean'],
    'Volume': ['sum', 'mean', 'std']
})

📊 金融数据聚合方法

📅 每周 OHLCV 数据（最近 8 周）：
              Open    High     Low   Close     Volume
Date                                                 
2025-08-03  213.79  214.61  201.27  202.15  319915100
2025-08-10  204.28  230.74  201.45  229.09  431826300
2025-08-17  227.92  235.12  224.76  231.59  295265800
2025-08-24  231.70  233.12  223.78  227.76  192241700
2025-08-31  226.48  233.41  224.69  232.14  194310800
2025-09-07  229.25  241.32  226.97  239.69  212923200
2025-09-14  239.30  240.15  225.95  234.07  304787000
2025-09-21  237.00  241.22  235.03  238.99  152555800


In [ ]:
# Different aggregation methods for different types of financial data
print("📊 金融数据聚合方法")
print("=" * 42)

# Select OHLCV data for aggregation
ohlcv_data = apple_stock[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()

# Weekly aggregation with appropriate methods for each column
weekly_ohlc = ohlcv_data.resample('W').agg({
    'Open': 'first',    # First trading day's opening price
    'High': 'max',      # Highest price during the week
    'Low': 'min',       # Lowest price during the week
    'Close': 'last',    # Last trading day's closing price
    'Volume': 'sum'     # Total volume traded during the week
})

print("\n📅 每周 OHLCV 数据（最近 8 周）：")
print(weekly_ohlc.tail(8).round(2))

# Monthly aggregation
monthly_stats = ohlcv_data.resample('M').agg({
    'Close': ['first', 'last', 'min', 'max', 'mean'],
    'Volume': ['sum', 'mean', 'std']
})

# Flatten column names
monthly_stats.columns = ['_'.join(col) for col in monthly_stats.columns]

print("\n📅 月度统计（最近 6 个月）：")
print(monthly_stats.tail(6).round(2))

# Calculate monthly returns and statistics
monthly_returns = monthly_stats['Close_last'].pct_change().dropna() * 100
monthly_volatility = (monthly_stats['Close_max'] - monthly_stats['Close_min']) / monthly_stats['Close_mean'] * 100

monthly_summary = pd.DataFrame({
    'Monthly_Return_%': monthly_returns,
    'Monthly_Volatility_%': monthly_volatility,
    'Avg_Daily_Volume_M': monthly_stats['Volume_mean'] / 1_000_000
})

print("\n📊 月度表现摘要:")
print(monthly_summary.tail(6).round(2))

📊 金融数据聚合方法

📅 每周 OHLCV 数据（最近 8 周）：
              Open    High     Low   Close     Volume
Date                                                 
2025-08-03  213.79  214.61  201.27  202.15  319915100
2025-08-10  204.28  230.74  201.45  229.09  431826300
2025-08-17  227.92  235.12  224.76  231.59  295265800
2025-08-24  231.70  233.12  223.78  227.76  192241700
2025-08-31  226.48  233.41  224.69  232.14  194310800
2025-09-07  229.25  241.32  226.97  239.69  212923200
2025-09-14  239.30  240.15  225.95  234.07  304787000
2025-09-21  237.00  241.22  235.03  238.99  152555800

📅 月度统计（最近 6 个月）：
            Close_first  Close_last  Close_min  Close_max  Close_mean  Volume_sum  Volume_mean   Volume_std
Date                                                                                                       
2025-04-30       222.65      211.98     172.00     223.34      200.69  1607318600  76538980.95  44317457.87
2025-05-31       212.80      200.62     195.05     212.80      203.63  1195728200  

## 5. 时间偏移和滞后变量

### 🎯 **您将在本节中学到什么：**

时间偏移是金融分析中的基本技术，允许您将当前值与过去值进行比较。这实现了动量分析、趋势检测和复杂的交易策略。

**关键知识点：**
- **滞后创建**：访问昨天、上周或上个月的值
- **领先创建**：访问未来值（用于对齐，而非预测！）
- **收益计算**：比较当前与历史价格
- **信号生成**：基于交叉和模式创建买入/卖出信号

**实际应用：**
- **动量分析**：股票是否延续昨天的趋势？
- **均值回归**：比较当前价格与历史平均
- **交易信号**：移动平均线交叉、突破模式
- **风险管理**：基于近期表现的投资组合再平衡

**🔄 核心概念：`shift()` 函数**

将 `shift()` 视为在时间上向前或向后移动数据：
- `shift(1)` = 将数据向前移动 1 个周期（创建昨天的值）
- `shift(-1)` = 将数据向后移动 1 个周期（创建明天的值）
- `shift(5)` = 将数据向前移动 5 个周期（为日数据创建上周的值）

时间偏移对于以下方面至关重要：
- 为预测模型创建滞后变量
- 计算周期变化
- 对齐数据以进行相关性分析

### 5.1 基本时间偏移

**🎯 理解时间偏移机制**

时间偏移将整个数据集在时间上向前或向后移动。这创建了与当前日期对齐的历史值的新列。

**视觉示例：**
```
原始数据：        偏移后 shift(1)：     偏移后 shift(-1)：
日期    收盘价       日期    收盘价       日期    收盘价
1月1日  100         1月1日  NaN         1月1日  110
1月2日  110         1月2日  100         1月2日  120  
1月3日  120         1月3日  110         1月3日  NaN
```

**🔍 关键观察：**
- **向前偏移 (+)**：创建**滞后**变量（历史值）
- **向后偏移 (-)**：创建**领先**变量（未来值）
- **NaN 值**：出现在不存在数据的边界
- **数据对齐**：相同索引，但偏移值

**⚠️ 关键概念 - 没有时间旅行！**
向后偏移 (`shift(-1)`) 不预测未来 - 它们用于：
- 对齐数据集
- 计算前瞻性指标
- 验证策略（使用已知的未来结果）

**常见金融应用：**
- `shift(1)`：昨天的收盘价
- `shift(5)`：上周的价格（用于周分析）
- `shift(21)`：上个月的价格（用于月比较）

让我们看看偏移的实际操作：

In [58]:
# Time shifting operations
print("📊 时间偏移操作")
print("=" * 30)

# Create a subset for demonstration
demo_data = apple_stock[['Close', 'Volume']].tail(10).copy()

print("\n📊 原始数据（最近 10 天）：")
print(demo_data.round(2))

# Forward shift (lag)
print("\n📊 向前偏移 (shift +1)：")
shifted_forward = demo_data.shift(1)
print(shifted_forward.round(2))

# Backward shift (lead)
print("\n📊 向后偏移 (shift -1)：")
shifted_backward = demo_data.shift(-1)
print(shifted_backward.round(2))

# Note: shifted data creates NaN values at the boundaries
print("\n⚠️ 注意：偏移在边界创建 NaN 值")
print(f"向前偏移：{shifted_forward.isnull().sum().sum()} 个 NaN 值") # why two sums?
print(f"向后偏移：{shifted_backward.isnull().sum().sum()} 个 NaN 值")

📊 时间偏移操作

📊 原始数据（最近 10 天）：
             Close    Volume
Date                        
2025-09-04  239.78  47549400
2025-09-05  239.69  54870400
2025-09-08  237.88  48999500
2025-09-09  234.35  66313900
2025-09-10  226.79  83440800
2025-09-11  230.03  50208600
2025-09-12  234.07  55824200
2025-09-15  236.70  42699500
2025-09-16  238.15  63421100
2025-09-17  238.99  46435200

📊 向前偏移 (shift +1)：
             Close      Volume
Date                          
2025-09-04     NaN         NaN
2025-09-05  239.78  47549400.0
2025-09-08  239.69  54870400.0
2025-09-09  237.88  48999500.0
2025-09-10  234.35  66313900.0
2025-09-11  226.79  83440800.0
2025-09-12  230.03  50208600.0
2025-09-15  234.07  55824200.0
2025-09-16  236.70  42699500.0
2025-09-17  238.15  63421100.0

📊 向后偏移 (shift -1)：
             Close      Volume
Date                          
2025-09-04  239.69  54870400.0
2025-09-05  237.88  48999500.0
2025-09-08  234.35  66313900.0
2025-09-09  226.79  83440800.0
2025-09-10  230.03  5020860

### 5.2 时间偏移的金融应用

**💰 实际时间偏移应用**

当应用于金融问题时，时间偏移从技术概念转变为强大的分析工具：

**1. 动量分析：**
- 比较今天价格与昨天：`价格 - 价格.shift(1)`
- 计算价格变化比率：`价格 / 价格.shift(1)`
- 识别趋势与区间市场

**2. 交易量确认：**
- 比较今天交易量与昨天：`交易量 / 交易量.shift(1)`
- 高相对交易量确认价格变动
- 低相对交易量表明变动较弱

**3. 收益持续性：**
- 今天收益与昨天收益的相关性
- 正相关 = 动量
- 负相关 = 均值回归

**📊 您将构建的关键指标：**

| 指标 | 公式 | 解释 |
|--------|---------|----------------|
| **价格变化** | `收盘价 - 收盘价.shift(1)` | 每日变化的美元金额 |
| **价格比率** | `收盘价 / 收盘价.shift(1)` | 今天与昨天的相对大小 |
| **交易量比率** | `交易量 / 交易量.shift(1)` | 相对交易活动 |
| **周变化** | `收盘价 - 收盘价.shift(5)` | 5 天价格变化 |

**🎯 模式识别：**
- **持续收益**：多个正日变化
- **交易量确认**：上涨日高交易量，下跌日低交易量
- **趋势强度**：更大的周变化表明更强的趋势

让我们构建这些分析工具：

In [66]:
# Practical financial applications of time shifting
print("📊 时间偏移的金融应用")
print("=" * 46)

# Create lagged variables for analysis
analysis_data = apple_stock[['Close', 'Volume', 'Daily_Return']].copy()

# Previous day's data (lag 1)
analysis_data['Close_Lag1'] = analysis_data['Close'].shift(1)
analysis_data['Volume_Lag1'] = analysis_data['Volume'].shift(1)
analysis_data['Return_Lag1'] = analysis_data['Daily_Return'].shift(1)

# Previous week's data (lag 5)
analysis_data['Close_Lag5'] = analysis_data['Close'].shift(5)
analysis_data['Return_Lag5'] = analysis_data['Daily_Return'].shift(5)

# Calculate relative changes
analysis_data['Price_Change_1d'] = analysis_data['Close'] - analysis_data['Close_Lag1']
analysis_data['Price_Change_5d'] = analysis_data['Close'] - analysis_data['Close_Lag5']
analysis_data['Volume_Ratio'] = analysis_data['Volume'] / analysis_data['Volume_Lag1']

print("\n📊 滞后变量示例（最近 5 天）：")
lag_example = analysis_data[[
    'Close', 'Close_Lag1', 'Price_Change_1d', 
    'Daily_Return', 'Return_Lag1', 'Volume_Ratio'
]].tail(5)
print(lag_example.round(3).to_string())

# Momentum analysis using lagged returns
momentum_data = analysis_data.dropna()

# Return persistence (today's return vs yesterday's return)
return_correlation = momentum_data['Daily_Return'].corr(momentum_data['Return_Lag1'])
print(f"\n📈 收益持续性（1 天滞后相关性）：{return_correlation:.4f}")

# Weekly momentum
weekly_momentum_corr = momentum_data['Daily_Return'].corr(momentum_data['Return_Lag5'])
print(f"📈 周动量（5 天滞后相关性）：{weekly_momentum_corr:.4f}")

# Volume-price relationship
volume_price_corr = momentum_data['Daily_Return'].corr(momentum_data['Volume_Ratio'])
print(f"📊 交易量-价格相关性：{volume_price_corr:.4f}")

# Trend identification using price changes
trend_signals = momentum_data.copy()
trend_signals['Trend_1d'] = np.where(trend_signals['Price_Change_1d'] > 0, '上涨', '下跌')
trend_signals['Trend_5d'] = np.where(trend_signals['Price_Change_5d'] > 0, '上涨', '下跌')

print("\n📊 近期趋势信号：")
trend_summary = trend_signals[['Price_Change_1d', 'Trend_1d', 'Price_Change_5d', 'Trend_5d']].tail(5)
print(trend_summary.round(2))

📊 时间偏移的金融应用

📊 滞后变量示例（最近 5 天）：
             Close  Close_Lag1  Price_Change_1d  Daily_Return  Return_Lag1  Volume_Ratio
Date                                                                                    
2025-09-11  230.03      226.79             3.24         0.014       -0.032         0.602
2025-09-12  234.07      230.03             4.04         0.018        0.014         1.112
2025-09-15  236.70      234.07             2.63         0.011        0.018         0.765
2025-09-16  238.15      236.70             1.45         0.006        0.011         1.485
2025-09-17  238.99      238.15             0.84         0.004        0.006         0.732

📈 收益持续性（1 天滞后相关性）：-0.0082
📈 周动量（5 天滞后相关性）：-0.0254
📊 交易量-价格相关性：-0.0176

📊 近期趋势信号：
            Price_Change_1d Trend_1d  Price_Change_5d Trend_5d
Date                                                          
2025-09-11             3.24       上涨            -9.75       下跌
2025-09-12             4.04       上涨            -5.62       下跌
2025-09-15  

## 6. 滚动和扩展窗口

### 🎯 **您将在本节中学到什么：**

滚动和扩展窗口是计算随时间变化的金融指标的基本工具。您将学习测量风险、表现和市场条件随时间的演变。

**关键知识点：**
- **滚动窗口**：固定周期的计算，在时间上“滑动”
- **扩展窗口**：从起点开始增长的计算
- **风险指标**：波动率、VaR、相关性、夏普比率
- **表现归因**：理解表现何时以及为何变化

**实际应用：**
- **风险管理**：实时监控投资组合波动率
- **表现评估**：随时间跟踪夏普比率
- **市场分析**：识别高/低波动率时期
- **投资组合优化**：理解变化的资产相关性

**🔄 滚动 vs 扩展 - 关键区别：**

| 窗口类型 | 计算 | 使用案例 | 示例 |
|-------------|-------------|----------|---------|
| **滚动** | 固定周期，向前滑动 | 当前条件 | 30 天波动率 |
| **扩展** | 从起点增长 | 累积指标 | 自成立以来的收益 |

**📊 视觉比较：**
```
滚动 3 天：    [1,2,3] → [2,3,4] → [3,4,5] → [4,5,6]
扩展：        [1] → [1,2] → [1,2,3] → [1,2,3,4] → [1,2,3,4,5]
```

对于计算随时间变化的金融指标至关重要。

### 6.1 滚动窗口计算

**📊 使用滚动窗口进行高级风险管理**

滚动窗口对于风险管理至关重要，因为它们提供**当前**风险估计，而不是历史平均值。金融市场变化迅速，风险度量需要适应。

**🎯 关键风险指标：**

**1. 滚动波动率**
- **目的**：衡量当前市场不确定性
- **常见窗口**：10d（短期）、30d（月度）、252d（年度）
- **解释**：更高波动率 = 更高风险 = 潜在更高收益

**2. 滚动相关性**
- **目的**：理解资产如何一起移动
- **应用**：投资组合多样化、对冲策略
- **关键洞察**：相关性随时间变化，尤其是在危机期间

**3. 滚动夏普比率**
- **目的**：风险调整后收益度量
- **公式**：`(收益 - 无风险利率) / 波动率`
- **解释**：更高夏普 = 更好的风险调整后表现


**📈 金融行业标准：**

| 指标 | 窗口 | 频率 | 使用案例 |
|--------|---------|-----------|----------|
| **波动率** | 30d | 每日 | 每日风险监控 |
| **相关性** | 63d | 每周 | 投资组合再平衡 |
| **夏普比率** | 252d | 每月 | 表现评估 |

让我们实现专业风险管理工具：

In [60]:
# Advanced rolling window calculations for risk management
print("📊 滚动窗口风险计算")
print("=" * 39)

# Load multi-asset data for portfolio analysis
multi_asset_prices = pd.read_csv('financial_close_prices.csv', index_col='Date', parse_dates=True)

# Select major assets for analysis
major_assets = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'SPY']
available_assets = [asset for asset in major_assets if asset in multi_asset_prices.columns]

if len(available_assets) >= 3:
    portfolio_prices = multi_asset_prices[available_assets].dropna()
    portfolio_returns = portfolio_prices.pct_change().dropna()

    print(f"\n📈 分析 {len(available_assets)} 个资产：{available_assets}")
    print(f"数据范围：{portfolio_returns.index[0].date()} 至 {portfolio_returns.index[-1].date()}")

    # Rolling volatility (different windows)
    windows = [10, 21, 63, 252]  # 2 weeks, 1 month, 3 months, 1 year

    volatility_data = pd.DataFrame(index=portfolio_returns.index)

    for window in windows:
        col_name = f'Vol_{window}d'
        volatility_data[col_name] = portfolio_returns['AAPL'].rolling(window=window).std() * np.sqrt(252) * 100

    print(f"\n📊 AAPL 滚动波动率（最近 10 天）：")
    print(volatility_data.tail(10).round(2))

    # Rolling correlations between assets
    if len(available_assets) >= 2:
        rolling_corr = portfolio_returns[available_assets[0]].rolling(window=63).corr(
            portfolio_returns[available_assets[1]]
        )

        print(f"\n📊 滚动 63 天相关性（{available_assets[0]} vs {available_assets[1]}）：")
        print(f"最新相关性：{rolling_corr.iloc[-1]:.3f}")
        print(f"平均相关性：{rolling_corr.mean():.3f}")
        print(f"相关性范围：{rolling_corr.min():.3f} 至 {rolling_corr.max():.3f}")

    # Rolling Sharpe ratio
    risk_free_rate = 0.02  # 2% annual risk-free rate
    daily_rf = risk_free_rate / 252

    for asset in available_assets[:3]:  # First 3 assets
        excess_returns = portfolio_returns[asset] - daily_rf
        rolling_sharpe = excess_returns.rolling(window=252).mean() / excess_returns.rolling(window=252).std() * np.sqrt(252)

        print(f"\n📊 {asset} - 252 天滚动夏普比率：")
        print(f"当前：{rolling_sharpe.iloc[-1]:.3f}")
        print(f"平均：{rolling_sharpe.mean():.3f}")


else:
    print("⚠️ 资产数据不足，无法进行投资组合分析")

📊 滚动窗口风险计算

📈 分析 5 个资产：['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'SPY']
数据范围：2020-09-22 至 2025-09-17

📊 AAPL 滚动波动率（最近 10 天）：
            Vol_10d  Vol_21d  Vol_63d  Vol_252d
Date                                           
2025-09-04    21.30    29.37    22.64     32.25
2025-09-05    20.77    25.02    22.49     32.13
2025-09-08    21.54    23.53    22.41     32.13
2025-09-09    23.45    19.79    22.49     32.16
2025-09-10    29.03    22.75    23.53     32.31
2025-09-11    29.91    22.99    23.26     32.34
2025-09-12    30.93    23.12    23.46     32.39
2025-09-15    31.31    23.41    23.27     32.39
2025-09-16    30.55    23.37    23.24     32.39
2025-09-17    23.88    23.32    22.98     32.39

📊 滚动 63 天相关性（AAPL vs MSFT）：
最新相关性：-0.001
平均相关性：0.608
相关性范围：-0.031 至 0.876

📊 AAPL - 252 天滚动夏普比率：
当前：0.333
平均：0.655

📊 MSFT - 252 天滚动夏普比率：
当前：0.757
平均：0.769

📊 GOOGL - 252 天滚动夏普比率：
当前：1.550
平均：0.634


### 6.2 扩展窗口分析

**📈 使用扩展窗口进行长期表现跟踪**

扩展窗口显示指标如何从数据集开始到每个时间点演变。它们回答诸如以下问题：*“自从我们开始投资以来，我们的夏普比率如何变化？”*

**🎯 关键应用：**

**1. 表现归因**
- 跟踪表现指标随时间变化
- 识别帮助或损害整体表现的时期
- 理解市场周期的影响

**2. 稳定性分析**
- 我们的风险指标是稳定还是仍在剧烈变化？
- 我们需要多少数据才能获得可靠估计？
- 我们的策略何时开始持续工作？

**3. 回撤分析**
- **最大回撤**：最大的峰谷下降
- **当前回撤**：我们离历史高点有多远？
- **恢复时间**：从重大损失中恢复需要多长时间？

**📊 基本扩展指标：**

| 指标 | 目的 | 洞察 |
|--------|---------|---------|
| **扩展收益** | 累积表现 | 自成立以来的总收益 |
| **扩展波动率** | 风险稳定性 | 风险估计是否稳定？ |
| **扩展夏普** | 风险调整后表现 | 整体策略质量 |
| **最大回撤** | 最坏情况 | 最大历史损失 |

**💡 为什么扩展窗口重要：**
- **监管报告**：许多法规要求自成立以来的指标
- **投资者沟通**：客户希望看到总表现
- **策略验证**：理解长期策略特征
- **风险预算**：基于历史经验规划最坏情况

让我们分析长期表现趋势：

In [61]:
# Demonstrate expanding window syntax
print("📈 扩展窗口语法演示")
print("=" * 45)

# Basic expanding operations
print("\n1️⃣ 基本扩展操作：")

# Get a small subset for clear demonstration
demo_data = apple_stock["Close"].head(10)
print("示例数据（前 10 天）：")
print(demo_data.round(2))

print("\n📊 扩展均值（累积平均）：")
expanding_mean = demo_data.expanding().mean()
print(expanding_mean.round(2))

print("\n📊 扩展和（累积和）：")
expanding_sum = demo_data.expanding().sum()
print(expanding_sum.round(2))

print("\n📊 扩展标准差：")
expanding_std = demo_data.expanding().std()
print(expanding_std.round(4))

print("\n📊 扩展最小/最大：")
expanding_min = demo_data.expanding().min()
expanding_max = demo_data.expanding().max()
print("扩展最小：")
print(expanding_min.round(2))
print("扩展最大：")
print(expanding_max.round(2))

# Show how window size grows
print("\n2️⃣ 窗口大小演示：")
print("扩展窗口从 1 增长到 N 个观测值：")
for i in range(5):
    window_size = i + 1
    subset = demo_data.iloc[:window_size]
    exp_mean = subset.expanding().mean().iloc[-1]
    manual_mean = subset.mean()
    print(f"第 {window_size} 天：窗口大小={window_size}，扩展均值={exp_mean:.2f}，手动均值={manual_mean:.2f}")

print("\n3️⃣ 与滚动窗口的关键区别：")
print("滚动窗口（固定大小）：使用最近 N 个观测值")
print("扩展窗口（增长大小）：使用从开始到当前点的所有观测值")
print("\n📈 扩展窗口语法演示完成！")

📈 扩展窗口语法演示

1️⃣ 基本扩展操作：
示例数据（前 10 天）：
Date
2020-09-21    107.08
2020-09-22    108.76
2020-09-23    104.20
2020-09-24    105.27
2020-09-25    109.22
2020-09-28    111.82
2020-09-29    110.98
2020-09-30    112.65
2020-10-01    113.60
2020-10-02    109.94
Name: Close, dtype: float64

📊 扩展均值（累积平均）：
Date
2020-09-21    107.08
2020-09-22    107.92
2020-09-23    106.68
2020-09-24    106.33
2020-09-25    106.90
2020-09-28    107.72
2020-09-29    108.19
2020-09-30    108.75
2020-10-01    109.29
2020-10-02    109.35
Name: Close, dtype: float64

📊 扩展和（累积和）：
Date
2020-09-21     107.08
2020-09-22     215.84
2020-09-23     320.03
2020-09-24     425.30
2020-09-25     534.52
2020-09-28     646.34
2020-09-29     757.32
2020-09-30     869.97
2020-10-01     983.57
2020-10-02    1093.51
Name: Close, dtype: float64

📊 扩展标准差：
Date
2020-09-21       NaN
2020-09-22    1.1899
2020-09-23    2.3070
2020-09-24    2.0114
2020-09-25    2.1694
2020-09-28    2.7927
2020-09-29    2.8305
2020-09-30    3.0588
2020-10-01  

In [67]:
# Expanding window analysis for long-term trends
print("📊 扩展窗口分析")
print("=" * 31)

# Use Apple stock data for expanding analysis
expanding_data = apple_stock[['Close', 'Daily_Return']].dropna()

# Expanding calculations (from start of data to current point)
expanding_data['Expanding_Mean_Return'] = expanding_data['Daily_Return'].expanding().mean() * 252 * 100
expanding_data['Expanding_Volatility'] = expanding_data['Daily_Return'].expanding().std() * np.sqrt(252) * 100
expanding_data['Expanding_Sharpe'] = (expanding_data['Daily_Return'].expanding().mean() - 0.02/252) / expanding_data['Daily_Return'].expanding().std() * np.sqrt(252)

# Expanding min/max (running extremes)
expanding_data['Expanding_Max_Price'] = expanding_data['Close'].expanding().max()
expanding_data['Expanding_Min_Price'] = expanding_data['Close'].expanding().min()
expanding_data['Drawdown'] = (expanding_data['Close'] - expanding_data['Expanding_Max_Price']) / expanding_data['Expanding_Max_Price'] * 100

expanding_data = expanding_data.dropna()
print(f"\n📊 扩展统计演变（每 100 天）：")
milestone_dates = expanding_data.iloc[::100]
display_cols = ['Expanding_Mean_Return', 'Expanding_Volatility', 'Expanding_Sharpe', 'Drawdown']
print(milestone_dates[display_cols].round(3).to_string())

print(f"\n📊 当前扩展统计：")
latest_stats = expanding_data[display_cols].iloc[-1]
print(f"年化收益：{latest_stats['Expanding_Mean_Return']:.2f}%")
print(f"年化波动率：{latest_stats['Expanding_Volatility']:.2f}%")
print(f"夏普比率：{latest_stats['Expanding_Sharpe']:.3f}")
print(f"当前回撤：{latest_stats['Drawdown']:.2f}%")

# Maximum drawdown analysis
max_drawdown = expanding_data['Drawdown'].min()
max_dd_date = expanding_data['Drawdown'].idxmin()
max_price_before_dd = expanding_data.loc[:max_dd_date, 'Expanding_Max_Price'].iloc[-1]
price_at_max_dd = expanding_data.loc[max_dd_date, 'Close']

print(f"\n📉 最大回撤分析：")
print(f"最大回撤：{max_drawdown:.2f}%")
print(f"最大回撤日期：{max_dd_date.date()}")
print(f"峰值价格：${max_price_before_dd:.2f}")
print(f"谷底价格：${price_at_max_dd:.2f}")

# Show recent expanding vs rolling metrics comparison
recent_data = expanding_data.tail(10)
rolling_vol = expanding_data['Daily_Return'].rolling(window=252).std() * np.sqrt(252) * 100

print(f"\n📊 扩展 vs 滚动波动率比较（最近 10 天）：")
volatility_comparison = pd.DataFrame({
    'Close': recent_data['Close'],
    'Expanding_Volatility': recent_data['Expanding_Volatility'],
    'Rolling_252d_Volatility': rolling_vol.tail(10)
})
print(volatility_comparison.round(2))

print("📈 扩展窗口分析完成！")

📊 扩展窗口分析

📊 扩展统计演变（每 100 天）：
            Expanding_Mean_Return  Expanding_Volatility  Expanding_Sharpe  Drawdown
Date                                                                               
2020-09-23               -330.501                64.725            -5.137    -4.195
2021-02-17                 49.211                33.947             1.391    -8.469
2021-07-12                 39.094                30.149             1.230    -0.420
2021-12-01                 38.065                27.370             1.318    -0.321
2022-04-26                 26.677                28.058             0.879   -13.741
2022-09-19                 22.118                30.019             0.670   -14.774
2023-02-10                 18.732                31.227             0.536   -16.423
2023-07-07                 24.759                29.832             0.763    -1.696
2023-11-28                 21.959                28.787             0.693    -2.821
2024-04-23                 16.102              

## 7. 金融时间序列中的缺失数据处理

**🔧 金融数据质量挑战**

金融数据是混乱的。市场因节假日关闭、系统故障、数据馈送存在缺口。专业分析师需要稳健的技术来处理这些现实。

**常见缺失数据场景：**
- **市场节假日**：无交易 = 无数据
- **系统中断**：技术故障造成缺口
- **交易暂停**：监管停止造成缺失时期
- **新上市**：股票在 IPO 前没有历史数据
- **退市证券**：股票停止交易并消失

**🎯 按数据类型的缺失数据策略：**

| 数据类型 | 最佳策略 | 理由 |
|-----------|---------------|-----------|
| **价格** | 向前填充 | 最后已知价格是最佳估计 |
| **交易量** | 零或中位数 | 无交易 = 零交易量，或使用典型交易量 |
| **收益** | 零 | 无交易时无价格变化 |
| **波动率** | 插值 | 已知值之间的平滑过渡 |

**⚠️ 关键考虑：**
- **不使用未来数据**：仅向前填充，从不向后用于分析
- **标记插值数据**：知道哪些观测值是真实 vs 估计
- **验证假设**：检查您的填充方法对特定情况是否有意义

让我们实现专业级数据清理：

Method 1: Forward Fill (ffill)
  - How it works: Uses the last valid observation to fill
  missing values
  - Best for: Price data where values tend to persist
  (e.g., stock prices over weekends)
  - Assumption: Missing values should equal the most
  recent known value
  - Example: If Monday's price is $100$ and Tuesday is
  missing, fills Tuesday with $100
  - Syntax: df['column'].fillna(method='ffill')

  Method 2: Linear Interpolation
  - How it works: Creates a straight line between known
  values to estimate missing points
  - Best for: Smooth price series where gradual changes
  are expected
  - Assumption: Missing values follow a linear trend
  between surrounding points
  - Example: If Monday=$100$ and Wednesday=$110$, fills
  Tuesday with $105$
  - Syntax: df['column'].interpolate(method='linear')

  Method 3: Time-Based Interpolation
  - How it works: Similar to linear but accounts for
  actual time intervals (weekends, holidays)
  - Best for: Financial data with irregular time spacing
  - Assumption: Changes occur proportionally to time
  elapsed
  - Example: Considers that weekend gaps should be
  weighted differently than daily gaps
  - Syntax: df['column'].interpolate(method='time')

In [63]:
# Advanced missing data handling for financial time series
print("📊 金融时间序列中的缺失数据处理")
print("=" * 52)

# Create sample data with missing values to demonstrate handling techniques
sample_data = apple_stock[['Close', 'Volume']].copy()

# Artificially introduce missing values for demonstration
np.random.seed(42)
missing_indices = np.random.choice(sample_data.index, size=50, replace=False)
sample_data.loc[missing_indices, 'Close'] = np.nan

# Add some random volume missing data
volume_missing = np.random.choice(sample_data.index, size=30, replace=False)
sample_data.loc[volume_missing, 'Volume'] = np.nan

print(f"\n📊 带缺失值的示例数据：")
print(f"总观测数：{len(sample_data)}")
print(f"缺失收盘价：{sample_data['Close'].isnull().sum()}")
print(f"缺失交易量数据：{sample_data['Volume'].isnull().sum()}")
print(f"缺失百分比：收盘价 {sample_data['Close'].isnull().mean()*100:.2f}%，交易量 {sample_data['Volume'].isnull().mean()*100:.2f}%")

# Method 1: Forward fill (common for prices)
sample_data['Close_ForwardFill'] = sample_data['Close'].fillna(method='ffill')

# Method 2: Linear interpolation (good for smooth price series)
sample_data['Close_Interpolated'] = sample_data['Close'].interpolate(method='linear')

# Method 3: Time-based interpolation (accounts for weekends/holidays)
sample_data['Close_TimeInterp'] = sample_data['Close'].interpolate(method='time')

# For volume data (different strategy - use median)
sample_data['Volume_MedianFill'] = sample_data['Volume'].fillna(sample_data['Volume'].median())

print(f"\n📊 缺失数据处理结果：")
methods = ['Close_ForwardFill', 'Close_Interpolated', 'Close_TimeInterp']
for method in methods:
    remaining_missing = sample_data[method].isnull().sum()
    print(f"{method}：{remaining_missing} 个缺失值剩余")

# Compare methods on a subset
comparison_data = sample_data.loc[missing_indices[:10]]
comparison_cols = ['Close', 'Close_ForwardFill', 'Close_Interpolated', 'Close_TimeInterp']
print(f"\n📊 缺失数据方法比较（示例）：")
print(comparison_data[comparison_cols].round(2))

📊 金融时间序列中的缺失数据处理

📊 带缺失值的示例数据：
总观测数：1254
缺失收盘价：50
缺失交易量数据：30
缺失百分比：收盘价 3.99%，交易量 2.39%

📊 缺失数据处理结果：
Close_ForwardFill：0 个缺失值剩余
Close_Interpolated：0 个缺失值剩余
Close_TimeInterp：0 个缺失值剩余

📊 缺失数据方法比较（示例）：
            Close  Close_ForwardFill  Close_Interpolated  Close_TimeInterp
Date                                                                      
2025-09-17    NaN             238.15              238.15            238.15
2021-02-16    NaN             132.10              129.89            128.56
2020-12-02    NaN             119.58              119.69            119.69
2020-12-18    NaN             125.40              125.18            125.29
2024-12-19    NaN             247.17              250.38            250.38
2023-12-12    NaN             191.56              193.93            193.93
2021-08-05    NaN             143.64              143.36            143.36
2022-08-15    NaN             169.43              169.88            170.11
2024-05-21    NaN             189.94              18

## 8. 总结和最佳实践

### 🎯 **您已完成的内容：**

恭喜！您已完成通过 pandas 时间序列分析进行金融分析的全面旅程。您现在拥有专业金融分析师日常使用的基本技能。

**🏆 您的新技能集：**

**数据管理技能：**
- 正确加载和构建金融数据
- 处理缺失数据和数据质量问题
- 使用不同时间频率和日历

**分析技能：**
- 计算收益、波动率和风险指标
- 实现技术指标和交易信号
- 执行投资组合分析和风险归因

**高级技术：**
- 使用滚动和扩展窗口进行动态分析
- 处理高频日内数据
- 创建和回测交易策略

**📈 从学生到专业分析师：**

您现在理解以下之间的区别：
- **学术金融**（干净数据，完美假设）
- **现实世界金融**（混乱数据，实际约束）

**💼 职业路径：**
- **投资组合经理**：优化资产配置和风险管理
- **量化分析师**：开发算法交易策略
- **风险经理**：监控和控制投资组合风险暴露
- **研究分析师**：分析证券和市场趋势
- **数据科学家**：将机器学习应用于金融问题

### 使用 Pandas 进行金融时间序列分析的关键要点

**🎓 掌握清单：**
- ✅ **数据加载**：您可以使用正确的日期时间索引加载金融数据
- ✅ **时间操作**：您可以切片、重采样和偏移时间序列数据
- ✅ **金融计算**：您可以计算收益、波动率和表现指标
- ✅ **技术分析**：您可以实现移动平均线、指标和交易信号
- ✅ **投资组合分析**：您可以分析多资产投资组合并衡量多样化
- ✅ **风险管理**：您可以计算 VaR、回撤和滚动风险指标
- ✅ **高级技术**：您可以处理缺失数据并使用高频数据

**🚀 金融分析之旅的下一步：**
1. **使用真实数据练习**：下载当前市场数据并应用这些技术
2. **学习领域专业知识**：理解您正在分析的金融市场和工具
3. **扩展您的工具包**：探索专业库，如 QuantLib、zipline 和 pyfolio
4. **构建完整系统**：创建端到端分析管道和自动化报告

In [64]:
# Summary of best practices and performance tips
print("📊 PANDAS 时间序列最佳实践总结")
print("=" * 48)

print("\n🏆 掌握的关键技术：")
print("" + "="*35)

techniques = {
    "1. 数据加载": [
        "✅ 使用 parse_dates=True 自动解析日期时间",
        "✅ 设置日期时间列作为索引以进行基于时间的操作",
        "✅ 为金融数据适当处理缺失值"
    ],
    "2. 基于时间的索引": [
        "✅ 利用 DateTimeIndex 进行直观的日期切片",
        "✅ 使用带日期时间组件的布尔索引（例如，df[df.index.year == 2024]）",
        "✅ 替代：使用完整日期字符串进行 .loc 索引（例如，'2024-01-01':'2024-01-31'）",
        "✅ 理解工作日与日历日操作"
    ],
    "3. 金融计算": [
        "✅ 使用 pct_change() 方法计算收益",
        "✅ 使用滚动窗口计算移动平均线和波动率",
        "✅ 应用扩展窗口计算累积统计"
    ],
    "4. 重采样": [
        "✅ 在频率之间转换（每日→每周→每月）",
        "✅ 选择适当的聚合方法（最后、均值、和）",
        "✅ 正确处理工作日频率"
    ],
    "5. 时间偏移": [
        "✅ 创建滞后变量进行分析",
        "✅ 使用时间偏移生成交易信号",
        "✅ 计算周期变化"
    ],
    "6. 高级分析": [
        "✅ 多资产投资组合分析",
        "✅ 风险指标（VaR、夏普比率、最大回撤）",
        "✅ 相关性和波动率聚类分析"
    ]
}

for category, items in techniques.items():
    print(f"\n{category}：")
    for item in items:
        print(f"  {item}")

print("\n💡 性能优化提示：")
print("" + "="*36)

perf_tips = [
    "🚀 使用向量化操作代替循环",
    "🚀 利用 pandas 内置金融函数",
    "🚀 可能时预分配 DataFrames",
    "🚀 使用适当的数据类型（float32 vs float64）",
    "🚀 在新列中缓存昂贵计算",
    "🚀 使用 .loc 和 .iloc 进行显式索引",
    "🚀 考虑对重复字符串使用分类数据"
]

for tip in perf_tips:
    print(f"  {tip}")

print("\n⚠️  要避免的常见陷阱：")
print("" + "="*30)

pitfalls = [
    "❌ 忘记处理金融数据中的缺失值",
    "❌ 在日期范围中未考虑周末/节假日",
    "❌ 在回测中使用前瞻信息",
    "❌ 忽略全球市场数据中的时区问题",
    "❌ 在分析前未验证数据质量",
    "❌ 混合不同资产类别而未标准化",
    "❌ 忘记年化收益和波动率"
]

for pitfall in pitfalls:
    print(f"  {pitfall}")

print("\n🎯 高级金融的下一步：")
print("" + "="*35)

next_steps = [
    "📈 学习量化金融库（例如，QuantLib、zipline）",
    "📈 探索用于金融预测的机器学习",
    "📈 研究高级风险模型（GARCH、VaR 模型）",
    "📈 实现算法交易策略",
    "📈 掌握投资组合优化技术",
    "📈 学习期权定价和衍生品分析",
    "📈 探索替代数据源（情绪、卫星数据）"
]

for step in next_steps:
    print(f"  {step}")

# Final statistics about what we've accomplished
print("\n📊 教程成就：")
print("" + "="*30)

print(f"📈 分析的金融数据集：6+（股票、ETF、加密货币、外汇）")
print(f"📈 演示的时间序列技术：15+")
print(f"📈 计算的技术指标：10+")
print(f"📈 掌握的 pandas DataFrame 操作：50+")
print(f"📈 计算的金融指标：20+")
print(f"📈 提供的代码示例：50+")

print("\n🎉 恭喜！")
print("您已完成关于 pandas 时间序列分析在金融中应用的全面教程。")
print("您现在具备进行专业金融数据分析的基本技能！")

print("\n" + "="*60)
print("📚 Pandas 时间序列分析在金融中应用教程结束")
print("="*60)

📊 PANDAS 时间序列最佳实践总结

🏆 掌握的关键技术：

1. 数据加载：
  ✅ 使用 parse_dates=True 自动解析日期时间
  ✅ 设置日期时间列作为索引以进行基于时间的操作
  ✅ 为金融数据适当处理缺失值

2. 基于时间的索引：
  ✅ 利用 DateTimeIndex 进行直观的日期切片
  ✅ 使用带日期时间组件的布尔索引（例如，df[df.index.year == 2024]）
  ✅ 替代：使用完整日期字符串进行 .loc 索引（例如，'2024-01-01':'2024-01-31'）
  ✅ 理解工作日与日历日操作

3. 金融计算：
  ✅ 使用 pct_change() 方法计算收益
  ✅ 使用滚动窗口计算移动平均线和波动率
  ✅ 应用扩展窗口计算累积统计

4. 重采样：
  ✅ 在频率之间转换（每日→每周→每月）
  ✅ 选择适当的聚合方法（最后、均值、和）
  ✅ 正确处理工作日频率

5. 时间偏移：
  ✅ 创建滞后变量进行分析
  ✅ 使用时间偏移生成交易信号
  ✅ 计算周期变化

6. 高级分析：
  ✅ 多资产投资组合分析
  ✅ 风险指标（VaR、夏普比率、最大回撤）
  ✅ 相关性和波动率聚类分析

💡 性能优化提示：
  🚀 使用向量化操作代替循环
  🚀 利用 pandas 内置金融函数
  🚀 可能时预分配 DataFrames
  🚀 使用适当的数据类型（float32 vs float64）
  🚀 在新列中缓存昂贵计算
  🚀 使用 .loc 和 .iloc 进行显式索引
  🚀 考虑对重复字符串使用分类数据

⚠️  要避免的常见陷阱：
  ❌ 忘记处理金融数据中的缺失值
  ❌ 在日期范围中未考虑周末/节假日
  ❌ 在回测中使用前瞻信息
  ❌ 忽略全球市场数据中的时区问题
  ❌ 在分析前未验证数据质量
  ❌ 混合不同资产类别而未标准化
  ❌ 忘记年化收益和波动率

🎯 高级金融的下一步：
  📈 学习量化金融库（例如，QuantLib、zipline）
  📈 探索用于金融预测的机器学习
  📈 研究高级风险模型（GARCH、VaR 模型）
  📈 实现算法交易策略
  📈 掌握投资组合优化技术
  📈 学习期权定价和衍生品分析
  📈 探索替代数据源（情绪、卫星数

---

## 📚 附加资源

### 文档和参考：
- [Pandas 时间序列文档](https://pandas.pydata.org/docs/user_guide/timeseries.html)
- [使用 Python 进行金融数据分析](https://github.com/PacktPublishing/Hands-On-Financial-Data-Analysis-with-Python)
- [量化金融资源](https://github.com/wilsonfreitas/awesome-quant)

### 高级金融库：
- **yfinance**：雅虎财经数据下载
- **quantlib**：量化金融库
- **zipline**：算法交易库
- **pyfolio**：投资组合和风险分析
- **ta-lib**：技术分析库

### 练习数据集：
- 雅虎财经 API
- Alpha Vantage API
- Quandl 金融数据
- FRED 经济数据